In [1]:
import h2o
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator

In [2]:
h2o.init()

H2O cluster uptime:,40 minutes 54 seconds 788 milliseconds
H2O cluster version:,3.5.0.99999
H2O cluster name:,ludirehak
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [3]:
from h2o.h2o import _locate # private function. used to find files within h2o git project directory.

# Import walking gait data
gait = h2o.upload_file(path=_locate("smalldata/glrm_test/subject01_walk1.csv"))
gait.describe()


Parse Progress: [##################################################] 100%
Rows:151 Cols:124

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2S,2-Byte Fractions,1,0.8,386 B,0.5
C4S,4-Byte Fractions,123,99.19355,82.6 KB,99.54594



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.38:54321,83.0 KB,151.0,1.0,124.0
mean,83.0 KB,151.0,1.0,124.0
min,83.0 KB,151.0,1.0,124.0
max,83.0 KB,151.0,1.0,124.0
stddev,0 B,0.0,0.0,0.0
total,83.0 KB,151.0,1.0,124.0


,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.57324,107.79163,600.01556,1028.72498,-152.53879,399.40009,1034.90125,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.25998,550.69415,684.75494,-133.588,435.01773,627.49988,-170.86086,319.34705,444.43289,157.69023,376.44724,403.25574,102.54192,293.89526,368.28577,165.6133,341.2023,438.80435,-172.96315,365.72482,375.95242,-120.97533,304.46347,393.67728,-173.38676,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,58.7473,-74.0361,211.85968,69.02702,-69.30095,185.39088,48.22646,-140.09074,254.42706,36.65646,-88.18266,627.38177,1342.85413,-35.20795,511.80032,1427.44507,164.48206,551.72424,1453.58936,-249.89265,485.94559,1227.90454,216.05853,531.12134,1217.64429,-266.60974,424.32767,1129.09949,242.29697,460.03223,1123.80298,-304.79358,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.59163,502.82971,886.58942,-300.87775,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,199.80693,36.02512,-153.13719,228.04898,42.9706,-54.88903,675.97009,1562.28418,44.15432,683.50897,1567.27209,-94.07465,614.13971,1740.38916,-19.48838
mean,1.25,613.813053113,1066.03966086,147.31687457,621.755521457,1053.96356967,-113.089856821,419.620581921,1059.09006815,21.623491457,604.893643709,752.113870596,198.394967219,673.92189702,697.923971391,108.985077152,611.822258874,633.86881755,191.199856954,585.490775166,756.997120795,-165.268507152,678.686755497,728.699455298,-105.36090649,595.940002185,651.790140662,-150.845648344,546.594923974,475.604231126,178.16108404,612.071029735,427.534634437,128.650221921,542.267098675,394.823727682,183.756735099,576.596701656,453

In [4]:
# Plot first row of data on x- vs. y-coordinate features
gait_row = gait[1,:].drop("Time")
gait_row_np = np.array(h2o.as_list(gait_row))
x_coords = range(0, gait_row_np.shape[1], 3)
y_coords = range(1, gait_row_np.shape[1], 3)

x_pts = gait_row_np[0,x_coords]
y_pts = gait_row_np[0,y_coords]
plt.plot(x_pts, y_pts, 'bo')

# Add feature labels to each point
feat_names = [nam[:-2] for nam in gait_row.col_names[1::3]]
for i in xrange(len(feat_names)):
    plt.annotate(feat_names[i], xy = [x_pts[i], y_pts[i]])
plt.title("Location of Body Parts at Time 0")
plt.xlabel("X-Coordinate Weight")
plt.ylabel("Y-Coordinate Weight")
plt.show()

In [5]:
# Basic GLRM using quadratic loss and no regularization (PCA)
model = H2OGeneralizedLowRankEstimator(k=10, loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=1000)
model.train(x=range(1,gait.ncol), training_frame=gait)
model.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2OGeneralizedLowRankEstimator :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1445997808349_97

Model Summary:


,number_of_iterations,final_step_size,final_objective_value
,16.0,0.0,309801.9




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 309801.943152
Misclassification Error (Categorical): 0.0

Scoring History:


,timestamp,duration,iteration,step_size,objective
,2015-10-27 19:44:27,0.221 sec,0.0,0.7,637672.3
,2015-10-27 19:44:27,0.252 sec,1.0,0.7,396983.5
,2015-10-27 19:44:27,0.268 sec,2.0,0.5,396983.5
,2015-10-27 19:44:28,0.278 sec,3.0,0.3,396983.5
,2015-10-27 19:44:28,0.289 sec,4.0,0.2,396983.5
,2015-10-27 19:44:28,0.298 sec,5.0,0.1,396983.5
,2015-10-27 19:44:28,0.307 sec,6.0,0.0,396983.5
,2015-10-27 19:44:28,0.317 sec,7.0,0.0,396983.5
,2015-10-27 19:44:28,0.326 sec,8.0,0.0,309801.9
,2015-10-27 19:44:28,0.336 sec,9.0,0.0,309801.9


In [6]:
# Plot objective function value each iteration
model_score = model.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
print model_score
plt.plot(model_score["iteration"], model_score["objective"])
plt.show()

                timestamp    duration  iteration  step_size      objective
0     2015-10-27 19:44:27   0.221 sec          0   0.666667  637672.264579
1     2015-10-27 19:44:27   0.252 sec          1   0.700000  396983.462707
2     2015-10-27 19:44:27   0.268 sec          2   0.466667  396983.462707
3     2015-10-27 19:44:28   0.278 sec          3   0.311111  396983.462707
4     2015-10-27 19:44:28   0.289 sec          4   0.207407  396983.462707
5     2015-10-27 19:44:28   0.298 sec          5   0.103704  396983.462707
6     2015-10-27 19:44:28   0.307 sec          6   0.034568  396983.462707
7     2015-10-27 19:44:28   0.317 sec          7   0.008642  396983.462707
8     2015-10-27 19:44:28   0.326 sec          8   0.009074  309801.937923
9     2015-10-27 19:44:28   0.336 sec          9   0.006049  309801.937923
10    2015-10-27 19:44:28   0.347 sec         10   0.004033  309801.937923
11    2015-10-27 19:44:28   0.356 sec         11   0.002689  309801.937923
12    2015-10-27 19:44:28

In [7]:
# Archetype to feature mapping (Y)
gait_y = model._model_json["output"]["archetypes"]
print gait_y

gait_y_np = np.array(model.archetypes())
x_coords = range(0, gait_y_np.shape[1], 3)
y_coords = range(1, gait_y_np.shape[1], 3)

# Plot archetypes on x- vs. y-coordinate features
for k in xrange(gait_y_np.shape[0]):
    x_pts = gait_y_np[k, x_coords]
    y_pts = gait_y_np[k, y_coords]
    plt.plot(x_pts, y_pts, 'bo')

    # Add feature labels to each point
    feat_names = [nam[:-1] for nam in gait_y.col_header[1::3]]
    for i in xrange(len(feat_names)):
        plt.annotate(feat_names[i], xy = [x_pts[i], y_pts[i]])
    plt.title("Feature Weights of Archetype " + str(k+1))
    plt.xlabel("X-Coordinate Weight")
    plt.ylabel("Y-Coordinate Weight")
    plt.show()


Archetypes:


,rasisx,rasisy,rasisz,lasisx,lasisy,lasisz,vsacralx,vsacraly,vsacralz,rthighupperx,rthighuppery,rthighupperz,rthighfrontx,rthighfronty,rthighfrontz,rthighrearx,rthighreary,rthighrearz,lthighupperx,lthighuppery,lthighupperz,lthighfrontx,lthighfronty,lthighfrontz,lthighrearx,lthighreary,lthighrearz,rshankupperx,rshankuppery,rshankupperz,rshankfrontx,rshankfronty,rshankfrontz,rshankrearx,rshankreary,rshankrearz,lshankupperx,lshankuppery,lshankupperz,lshankfrontx,lshankfronty,lshankfrontz,lshankrearx,lshankreary,lshankrearz,rheelx,rheely,rheelz,rmidfootsupx,rmidfootsupy,rmidfootsupz,rmidfootlatx,rmidfootlaty,rmidfootlatz,rtoetipx,rtoetipy,rtoetipz,lheelx,lheely,lheelz,lmidfootsupx,lmidfootsupy,lmidfootsupz,lmidfootlatx,lmidfootlaty,lmidfootlatz,ltoetipx,ltoetipy,ltoetipz,sternumx,sternumy,sternumz,racromiumx,racromiumy,racromiumz,lacromiumx,lacromiumy,lacromiumz,rbicepx,rbicepy,rbicepz,lbicepx,lbicepy,lbicepz,relbowx,relbowy,relbowz,lelbowx,lelbowy,lelbowz,rwristmedx,rwristmedy,rwristmedz,rwristlatx,rwristlaty,rwristlatz,lwristmedx,lwristmedy,lwristmedz,lwristlatx,lwristlaty,lwristlatz,rtoelatx,rtoelaty,rtoelatz,rtoemedx,rtoemedy,rtoemedz,ltoelatx,ltoelaty,ltoelatz,ltoemedx,ltoemedy,ltoemedz,rtemplex,rtempley,rtemplez,ltemplex,ltempley,ltemplez,topheadx,topheady,topheadz
Arch1,611.1,1052.5,164.3,634.2,1045.5,-96.1,424.0,1050.8,22.5,510.8,738.9,210.5,568.6,661.6,127.8,476.2,622.5,202.3,653.4,746.0,-147.7,750.2,740.4,-81.2,690.9,644.3,-132.8,351.0,479.0,189.8,402.8,408.7,141.1,319.9,398.3,193.9,741.7,443.0,-125.8,800.1,405.2,-72.6,731.5,396.8,-134.3,101.5,161.7,102.4,232.3,107.0,73.9,211.7,90.4,180.0,337.6,39.3,116.7,741.6,68.9,-44.1,904.8,95.6,-17.2,891.6,87.3,-114.0,1018.9,96.0,-45.6,650.7,1353.2,7.4,555.8,1450.4,215.5,582.5,1462.6,-202.4,578.7,1251.8,268.2,571.5,1225.6,-232.1,548.6,1147.0,305.8,509.6,1136.3,-266.7,736.9,957.2,276.7,708.5,934.1,331.6,608.7,905.1,-223.1,560.4,895.5,-280.1,273.5,35.0,197.7,293.4,39.9,86.3,956.9,74.6,-122.7,970.7,84.4,-20.6,724.5,1571.6,91.6,737.6,1573.9,-46.8,743.3,1748.9,30.3
Arch2,620.4,1083.4,128.3,636.8,1071.1,-132.4,429.3,1072.0,-3.3,635.2,762.7,195.8,719.7,710.5,118.8,653.9,642.1,198.3,605.0,764.4,-177.6,695.2,735.6,-110.1,611.3,655.9,-156.1,567.1,521.0,185.0,601.5,436.3,143.8,514.2,452.2,189.3,608.3,441.4,-145.6,652.3,392.2,-87.9,587.0,398.4,-151.6,258.2,305.0,78.0,352.4,191.4,72.8,312.7,196.1,171.4,402.1,65.5,131.7,505.3,73.7,-47.8,677.6,74.5,-17.2,662.7,69.9,-115.4,791.2,51.1,-46.9,649.0,1385.7,-23.9,538.9,1483.4,177.5,591.4,1487.9,-239.7,536.0,1285.1,233.4,625.5,1248.0,-255.3,486.0,1186.5,265.3,584.7,1147.6,-290.6,609.5,944.0,280.8,564.2,935.9,329.1,703.8,930.8,-229.8,663.1,915.8,-291.4,338.2,111.4,196.6,374.9,102.9,93.2,727.9,44.0,-123.2,740.7,52.5,-21.2,711.7,1610.7,55.2,728.1,1612.2,-83.3,720.2,1788.3,-9.6
Arch3,629.1,1053.9,141.6,620.4,1037.8,-119.5,427.6,1042.4,33.1,681.0,748.9,192.3,753.1,710.7,94.3,714.8,631.4,180.3,515.3,746.1,-172.3,607.5,692.3,-125.7,497.6,639.2,-160.2,699.9,454.6,172.1,777.4,428.1,112.3,724.4,370.8,178.0,404.8,452.3,-162.3,411.6,381.1,-110.4,360.8,432.6,-166.2,671.2,66.9,96.2,814.7,81.4,64.4,805.4,64.6,171.5,958.2,63.2,106.9,96.4,222.8,-65.2,220.2,115.3,-36.8,195.4,106.4,-130.5,277.9,39.9,-54.4,648.5,1356.7,24.6,538.1,1435.4,234.2,588.1,1476.4,-180.5,528.0,1234.3,272.8,621.1,1240.4,-216.2,475.5,1133.4,293.4,582.5,1141.5,-260.7,610.4,898.2,295.2,563.7,883.3,339.2,718.9,933.5,-207.1,681.5,916.3,-269.8,892.9,43.9,186.3,903.3,55.1,76.2,219.6,53.8,-132.8,247.1,57.4,-31.9,703.4,1589.4,117.6,716.7,1597.3,-20.8,687.8,1765.4,59.6
Arch4,614.9,1088.0,112.6,625.0,1064.4,-146.7,421.9,1073.6,-12.6,675.2,783.6,182.6,762.0,745.8,98.4,714.4,667.5,182.4,550.6,764.0,-188.3,638.5,722.2,-122.5,541.6,654.4,-166.1,657.2,521.6,169.0,718.1,455.2,128.0,634.1,439.0,176.5,508.0,441.0,-155.0,552.6,389.1,-101.8,484.4,399.5,-160.0,448.7,195.2,73.5,583.1,141.4,71.0,549.4,132.1,170.5,688.9,56.0,136.0,388.8,81.0,-57.2,556.1,76.3,-28.5,541.1,65.4,-124.8,666.3,48.1,-56.2,647.0,1383.1,-29.7,535.2,1

In [8]:
# Projection into archetype space (X)
x_key = model._model_json["output"]["representation_name"]
gait_x = h2o.get_frame(x_key)
gait_x.show()

time_np = np.array(h2o.as_list(gait["Time"]))
gait_x_np = np.array(h2o.as_list(gait_x))

# Plot archetypes over time
lines = []
for i in xrange(gait_x_np.shape[1]):
    lines += plt.plot(time_np, gait_x_np[:,i], '-')
plt.title("Archetypes over Time")
plt.xlabel("Time")
plt.ylabel("Archetypal Projection")
plt.legend(lines, gait_x.col_names)
plt.show()

Arch1,Arch2,Arch3,Arch4,Arch5,Arch6,Arch7,Arch8,Arch9,Arch10
0.731507,-0.669527,-0.576851,0.684254,0.530546,0.349798,-0.300091,0.680207,0.160855,-0.574808
0.831345,-0.632144,-0.509253,0.599608,0.367028,0.298258,-0.236882,0.58155,0.190056,-0.475987
0.927611,-0.596205,-0.416523,0.514627,0.202298,0.227356,-0.171733,0.474841,0.226304,-0.377064
0.968412,-0.581962,-0.339319,0.45547,0.0651782,0.168298,-0.11049,0.386156,0.301602,-0.303684
0.953375,-0.594609,-0.285284,0.420949,-0.0473836,0.132213,-0.0598403,0.304204,0.422379,-0.238353
0.895114,-0.598648,-0.230742,0.396169,-0.122648,0.0975691,-0.0258822,0.249581,0.553998,-0.208168
0.758408,-0.640395,-0.216997,0.411987,-0.167509,0.0877752,0.00844827,0.229631,0.74639,-0.212649
0.589719,-0.660098,-0.208855,0.425481,-0.174692,0.0901791,0.0135659,0.218862,0.933599,-0.223489
0.389964,-0.660729,-0.207493,0.424605,-0.149256,0.0987919,-0.00382048,0.222449,1.12259,-0.233346
0.172431,-0.626891,-0.203323,0.419041,-0.0925958,0.10348,-0.0342347,0.24454,1.28358,-0.261784


In [9]:
# Reconstruct data from X and Y
pred = model.predict(gait)
pred.head()

Expr(op='rows',id=None,ast=(Expr(op='cols_py',id=None,ast=(Expr(op='Frame',id=u'predictions_bebc_GLRM_model_python_1445997808349_97_on_Key_Frame__upload_a032b0fa83740187a995804820f74ff3.hex',ast=(False,),nrows=151,ncols=123,data=<class 'collections.OrderedDict'>), slice(None, 123, None)),nrows=None,ncols=None,data=<type 'NoneType'>), slice(None, 10, None)),nrows=None,ncols=None,data=<type 'NoneType'>)

In [10]:
# Plot original and reconstructed L.Acromium.X over time
lacro_np = np.array(h2o.as_list(gait["L.Acromium.X"]))
lacro_pred_np = np.array(h2o.as_list(pred["reconstr_L.Acromium.X"]))
line_orig = plt.plot(time_np, lacro_np, '-')
line_imp = plt.plot(time_np, lacro_pred_np, '-')

plt.title("Position of Left Acromium over Time")
plt.xlabel("Time")
plt.ylabel("X-Coordinate of Left Acromium")
blue_patch = mpatches.Patch(color = 'blue', label = 'Original')
green_patch = mpatches.Patch(color = 'green', label='Imputed')
plt.legend([blue_patch, green_patch], ["Original", "Imputed"])
plt.show()

In [11]:
# Import walking gait data with missing values
gait_miss = h2o.upload_file(_locate("smalldata/glrm_test/subject01_walk1_miss15.csv"))
gait_miss.describe()


Parse Progress: [##################################################] 100%
Rows:151 Cols:124

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C2S,2-Byte Fractions,1,0.8,386 B,0.5
C4S,4-Byte Fractions,123,99.19355,82.6 KB,99.54594



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.38:54321,83.0 KB,151.0,1.0,124.0
mean,83.0 KB,151.0,1.0,124.0
min,83.0 KB,151.0,1.0,124.0
max,83.0 KB,151.0,1.0,124.0
stddev,0 B,0.0,0.0,0.0
total,83.0 KB,151.0,1.0,124.0


,Time,R.ASIS.X,R.ASIS.Y,R.ASIS.Z,L.ASIS.X,L.ASIS.Y,L.ASIS.Z,V.Sacral.X,V.Sacral.Y,V.Sacral.Z,R.Thigh.Upper.X,R.Thigh.Upper.Y,R.Thigh.Upper.Z,R.Thigh.Front.X,R.Thigh.Front.Y,R.Thigh.Front.Z,R.Thigh.Rear.X,R.Thigh.Rear.Y,R.Thigh.Rear.Z,L.Thigh.Upper.X,L.Thigh.Upper.Y,L.Thigh.Upper.Z,L.Thigh.Front.X,L.Thigh.Front.Y,L.Thigh.Front.Z,L.Thigh.Rear.X,L.Thigh.Rear.Y,L.Thigh.Rear.Z,R.Shank.Upper.X,R.Shank.Upper.Y,R.Shank.Upper.Z,R.Shank.Front.X,R.Shank.Front.Y,R.Shank.Front.Z,R.Shank.Rear.X,R.Shank.Rear.Y,R.Shank.Rear.Z,L.Shank.Upper.X,L.Shank.Upper.Y,L.Shank.Upper.Z,L.Shank.Front.X,L.Shank.Front.Y,L.Shank.Front.Z,L.Shank.Rear.X,L.Shank.Rear.Y,L.Shank.Rear.Z,R.Heel.X,R.Heel.Y,R.Heel.Z,R.Midfoot.Sup.X,R.Midfoot.Sup.Y,R.Midfoot.Sup.Z,R.Midfoot.Lat.X,R.Midfoot.Lat.Y,R.Midfoot.Lat.Z,R.Toe.Tip.X,R.Toe.Tip.Y,R.Toe.Tip.Z,L.Heel.X,L.Heel.Y,L.Heel.Z,L.Midfoot.Sup.X,L.Midfoot.Sup.Y,L.Midfoot.Sup.Z,L.Midfoot.Lat.X,L.Midfoot.Lat.Y,L.Midfoot.Lat.Z,L.Toe.Tip.X,L.Toe.Tip.Y,L.Toe.Tip.Z,Sternum.X,Sternum.Y,Sternum.Z,R.Acromium.X,R.Acromium.Y,R.Acromium.Z,L.Acromium.X,L.Acromium.Y,L.Acromium.Z,R.Bicep.X,R.Bicep.Y,R.Bicep.Z,L.Bicep.X,L.Bicep.Y,L.Bicep.Z,R.Elbow.X,R.Elbow.Y,R.Elbow.Z,L.Elbow.X,L.Elbow.Y,L.Elbow.Z,R.Wrist.Med.X,R.Wrist.Med.Y,R.Wrist.Med.Z,R.Wrist.Lat.X,R.Wrist.Lat.Y,R.Wrist.Lat.Z,L.Wrist.Med.X,L.Wrist.Med.Y,L.Wrist.Med.Z,L.Wrist.Lat.X,L.Wrist.Lat.Y,L.Wrist.Lat.Z,R.Toe.Lat.X,R.Toe.Lat.Y,R.Toe.Lat.Z,R.Toe.Med.X,R.Toe.Med.Y,R.Toe.Med.Z,L.Toe.Lat.X,L.Toe.Lat.Y,L.Toe.Lat.Z,L.Toe.Med.X,L.Toe.Med.Y,L.Toe.Med.Z,R.Temple.X,R.Temple.Y,R.Temple.Z,L.Temple.X,L.Temple.Y,L.Temple.Z,Top.Head.X,Top.Head.Y,Top.Head.Z
type,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,0.0,588.90717,1042.57324,107.79163,600.01556,1029.25427,-152.53879,399.40009,1034.90125,-21.44083,481.83603,732.70135,169.71159,535.63422,657.21033,74.20708,443.03299,613.54547,168.15961,468.19415,737.72083,-194.25998,550.69415,684.75494,-133.588,435.01773,627.49988,-170.86086,319.34705,445.08429,157.69023,376.44724,403.25574,102.54192,295.04089,368.36661,165.6133,341.2023,438.80435,-172.96315,365.72482,375.95242,-120.97533,304.46347,393.67728,-173.38676,68.60076,56.43383,59.948,190.98845,76.90237,57.86759,157.32503,62.79,159.97461,242.90858,33.52863,102.0894,76.37951,60.01719,-74.0361,211.85968,70.39124,-69.30095,185.39088,48.22646,-140.09074,254.42706,36.65646,-88.18266,627.38177,1342.85413,-35.20795,511.80032,1427.44507,164.48206,551.72424,1453.58936,-249.89265,485.94559,1227.90454,216.05853,531.12134,1217.64429,-266.60974,424.32767,1129.49243,242.4415,460.20703,1123.80298,-304.79358,535.87439,889.81964,262.1105,486.99316,876.53882,304.71069,554.35272,895.92426,-240.59163,503.10611,886.58942,-300.87775,179.7708,33.33099,181.24631,218.04131,36.96362,71.79319,202.74066,36.02512,-153.13719,228.04898,42.9706,-54.88903,675.97009,1562.28418,44.15432,683.50897,1567.27209,-94.07465,614.13971,1740.38916,-19.48838
mean,1.25,614.521937381,1065.67421107,147.703810859,621.468577752,1053.88708992,-113.122046299,419.961478582,1058.93961659,21.1173823529,602.023772652,751.727318268,198.126564524,676.998435725,698.116475255,107.988446202,607.493851145,634.098869191,191.194278051,589.7071056,756.634831575,-164.439363308,676.351652422,728.968314375,-105.687799286,598.001710242,651.426971145,-150.982097813,543.651496589,476.831541692,177.902406439,612.248566296,427.129153607,128.558000234,550.95748125,395.283420826,183.200654918,578.969913643,

In [12]:
# Basic GLRM using quadratic loss and no regularization

model2 = H2OGeneralizedLowRankEstimator(k=10, init="SVD", svd_method = "GramSVD", loss="Quadratic", regularization_x="None", regularization_y="None", max_iterations=2000, min_step_size=1e-6)
model2.train(x=range(1,gait_miss.ncol), training_frame=gait_miss, validation_frame=gait)
model2.show()


glrm Model Build Progress: [##################################################] 100%
Model Details
H2OGeneralizedLowRankEstimator :  Beta - Generalized Low Rank Model
Model Key:  GLRM_model_python_1445997808349_100

Model Summary:


,number_of_iterations,final_step_size,final_objective_value
,2000.0,0.0,122299.7




ModelMetricsGLRM: glrm
** Reported on train data. **

MSE: NaN
Sum of Squared Error (Numeric): 122299.726469
Misclassification Error (Categorical): 0.0

ModelMetricsGLRM: glrm
** Reported on validation data. **

MSE: NaN
Sum of Squared Error (Numeric): 153530.207952
Misclassification Error (Categorical): 0.0

Scoring History:


,timestamp,duration,iteration,step_size,objective
,2015-10-27 19:44:37,0.324 sec,0.0,0.7,4709528.9
,2015-10-27 19:44:37,0.332 sec,1.0,0.4,4709528.9
,2015-10-27 19:44:37,0.340 sec,2.0,0.2,4709528.9
,2015-10-27 19:44:37,0.348 sec,3.0,0.1,4709528.9
,2015-10-27 19:44:37,0.357 sec,4.0,0.0,4709528.9
---,---,---,---,---,---
,2015-10-27 19:45:06,28.769 sec,1995.0,0.0,122316.1
,2015-10-27 19:45:06,28.791 sec,1996.0,0.0,122310.5
,2015-10-27 19:45:06,28.811 sec,1997.0,0.0,122306.5
,2015-10-27 19:45:06,28.832 sec,1998.0,0.0,122303.0


In [13]:
# Plot objective function value each iteration
model2_score = model2.score_history()
plt.xlabel("Iteration")
plt.ylabel("Objective")
plt.title("Objective Function Value per Iteration")
plt.plot(model2_score["iteration"], model2_score["objective"])
plt.show()

In [14]:
# Impute missing data from X and Y
pred2 = model2.predict(gait_miss)
pred2.head()

Expr(op='rows',id=None,ast=(Expr(op='cols_py',id=None,ast=(Expr(op='Frame',id=u'predictions_965e_GLRM_model_python_1445997808349_100_on_Key_Frame__upload_aeb490953d9d5e70cf3ea0145d774466.hex',ast=(False,),nrows=151,ncols=123,data=<class 'collections.OrderedDict'>), slice(None, 123, None)),nrows=None,ncols=None,data=<type 'NoneType'>), slice(None, 10, None)),nrows=None,ncols=None,data=<type 'NoneType'>)

In [29]:
# Plot original and imputed L.Acromium.X over time
lacro_pred_np2 = np.array(h2o.as_list(pred2["reconstr_L.Acromium.X"]))
plt.plot(time_np, lacro_np, 'b-')
plt.plot(time_np, lacro_pred_np2, 'g-')

# Mark points where training data contains missing values
idx_miss = zip(*gait_miss["L.Acromium.X"].isna().which().as_data_frame().values.tolist())
plt.plot(time_np[idx_miss], lacro_np[idx_miss], "o", marker = "x", ms = 8, mew = 1.5, mec = "r")

plt.title("Position of Left Acromium over Time")
plt.xlabel("Time")
plt.ylabel("X-Coordinate of Left Acromium")
blue_patch = mpatches.Patch(color = 'blue', label = 'Original')
green_patch = mpatches.Patch(color = 'green', label = 'Imputed')
red_patch = mpatches.Patch(color = 'red', label = "Missing")
plt.legend([blue_patch, green_patch, red_patch], ["Original", "Imputed", "Missing"])
plt.show()